In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns #vẽ biểu đồ
#!pip install --upgrade scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import svm
!pip install openpyxl --upgrade #để xử lý file excel
from sklearn.model_selection import cross_val_score

In [ ]:
import sklearn
print(sklearn.__version__)

1.5.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/outlad(gop)/'

/content/drive/MyDrive/Colab Notebooks/outlad(gop)


In [ ]:
!ls

assessments.csv		     studentAssessment.csv    sum_click_by_activity_by_day.csv
Classification_Outlad.ipynb  studentInfo.csv	      sum_click_by_activity_by_week.csv
courses.csv		     studentRegistration.csv  sum_clicks_by_student_by_day.csv
OULAD.names		     studentVle.csv	      vle.csv


# Đọc dữ liệu

In [ ]:
df_studentvle = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/outlad(gop)/studentVle.csv')
df_studentinfo = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/outlad(gop)/studentInfo.csv')
df_studentvle.head()

,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1


In [ ]:
df_vle = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/outlad(gop)/vle.csv')
df_vle.head()

,id_site,code_module,code_presentation,activity_type,week_from,week_to
0,546943,AAA,2013J,resource,?,?
1,546712,AAA,2013J,oucontent,?,?
2,546998,AAA,2013J,resource,?,?
3,546888,AAA,2013J,url,?,?
4,547035,AAA,2013J,resource,?,?


# Tiền xử lý dữ liệu

In [ ]:
#bổ sung thêm trường activity_type vào bảng studentvle dựa trên bảng vle
sdvle_with_activity = df_studentvle.merge(df_vle[['id_site', 'activity_type']], on='id_site', how='left')
sdvle_with_activity.head()

,code_module,code_presentation,id_student,id_site,date,sum_click,activity_type
0,AAA,2013J,28400,546652,-10,4,forumng
1,AAA,2013J,28400,546652,-10,1,forumng
2,AAA,2013J,28400,546652,-10,1,forumng
3,AAA,2013J,28400,546614,-10,11,homepage
4,AAA,2013J,28400,546714,-10,1,oucontent


In [ ]:
# Tính số click theo ngày. Nhóm theo các trường 'code_module', 'code_presentation', 'id_student', 'date', and 'activity_type'
sum_click_by_activity_by_day = sdvle_with_activity.groupby(['code_module', 'code_presentation', 'id_student', 'date', 'activity_type'])['sum_click'].sum().reset_index()

# Display the resulting DataFrame
print(sum_click_by_activity_by_day.head())

  code_module code_presentation  id_student  date activity_type  sum_click
0         AAA             2013J       11391    -5       forumng          2
1         AAA             2013J       11391    -5      homepage          7
2         AAA             2013J       11391    -5     oucontent         78
3         AAA             2013J       11391    -5       subpage         11
4         AAA             2013J       11391     0       forumng         15


In [ ]:
#Tính toán click theo ngày, nhóm theo 'code_module', 'code_presentation', 'id_student', 'date' và theo từng hoạt động (20 hđ)
sum_click_by_activity_by_day = sum_click_by_activity_by_day.pivot_table(
    index=['code_module', 'code_presentation', 'id_student', 'date'],
    columns='activity_type',
    values='sum_click',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Hiển thị DataFrame kết quả
print(sum_click_by_activity_by_day.head())

activity_type code_module code_presentation  id_student  date  dataplus  \
0                     AAA             2013J       11391    -5         0   
1                     AAA             2013J       11391     0         0   
2                     AAA             2013J       11391     1         0   
3                     AAA             2013J       11391     2         0   
4                     AAA             2013J       11391     6         0   

activity_type  dualpane  externalquiz  folder  forumng  glossary  ...  \
0                     0             0       0        2         0  ...   
1                     0             0       0       15         0  ...   
2                     0             0       0        3         0  ...   
3                     0             0       0        0         0  ...   
4                     0             0       0        0         0  ...   

activity_type  ouelluminate  ouwiki  page  questionnaire  quiz  \
0                         0       0     0   

In [ ]:
#Xuất file
output_csv_path='./sum_click_by_activity_by_day.csv'
sum_click_by_activity_by_day.to_csv(output_csv_path, index=False)
#

In [ ]:
# Thêm cột 'week' bằng cách chia cột 'date' cho 7 và lấy giá trị nguyên dương
sum_click_by_activity_by_day['week'] = ((sum_click_by_activity_by_day['date']-1) // 7) + 1

# Xóa cột 'date'
sum_click_by_activity_by_day = sum_click_by_activity_by_day.drop(columns=['date'])

# Thống kê dữ liệu theo tuần, nhóm theo 'code_module', 'code_presentation', 'id_student', và 'week'
sum_click_by_activity_by_week = sum_click_by_activity_by_day.groupby(
    ['code_module', 'code_presentation', 'id_student', 'week']
).sum().reset_index()

# Hiển thị DataFrame kết quả
print(sum_click_by_activity_by_week.head())

activity_type code_module code_presentation  id_student  week  dataplus  \
0                     AAA             2013J       11391     0         0   
1                     AAA             2013J       11391     1         0   
2                     AAA             2013J       11391     2         0   
3                     AAA             2013J       11391     3         0   
4                     AAA             2013J       11391     5         0   

activity_type  dualpane  externalquiz  folder  forumng  glossary  ...  \
0                     0             0       0       17         0  ...   
1                     0             0       0        3         0  ...   
2                     0             0       0        0         0  ...   
3                     0             0       0       35         0  ...   
4                     0             0       0        2         0  ...   

activity_type  ouelluminate  ouwiki  page  questionnaire  quiz  \
0                         0       0     0   

In [ ]:
#trộn trường final_result từ bảng studentinfo dựa trên code_module, code_presentation, id_student
sum_click_by_activity_by_week = sum_click_by_activity_by_week.merge(df_studentinfo[['code_module','code_presentation','id_student', 'final_result']], on=['code_module', 'code_presentation', 'id_student'], how='left')
sum_click_by_activity_by_week.head()

,code_module,code_presentation,id_student,week,dataplus,dualpane,externalquiz,folder,forumng,glossary,...,ouwiki,page,questionnaire,quiz,repeatactivity,resource,sharedsubpage,subpage,url,final_result
0,AAA,2013J,11391,0,0,0,0,0,17,0,...,0,0,0,0,0,3,0,11,0,Pass
1,AAA,2013J,11391,1,0,0,0,0,3,0,...,0,0,0,0,0,5,0,8,1,Pass
2,AAA,2013J,11391,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,Pass
3,AAA,2013J,11391,3,0,0,0,0,35,0,...,0,0,0,0,0,1,0,1,0,Pass
4,AAA,2013J,11391,5,0,0,0,0,2,0,...,0,0,0,0,0,0,0,1,0,Pass


In [ ]:
output_csv_path='./sum_click_by_activity_by_week(final_result).csv'
sum_click_by_activity_by_week.to_csv(output_csv_path, index=False)

In [ ]:
#Lọc ra những dữ liệu có final_result là Pass hoặc Distinction
pass_df = sum_click_by_activity_by_week[sum_click_by_activity_by_week['final_result'].isin(['Pass', 'Distinction'])]
print(pass_df.head())

  code_module code_presentation  id_student  week  dataplus  dualpane  \
0         AAA             2013J       11391     0         0         0   
1         AAA             2013J       11391     1         0         0   
2         AAA             2013J       11391     2         0         0   
3         AAA             2013J       11391     3         0         0   
4         AAA             2013J       11391     5         0         0   

   externalquiz  folder  forumng  glossary  ...  ouwiki  page  questionnaire  \
0             0       0       17         0  ...       0     0              0   
1             0       0        3         0  ...       0     0              0   
2             0       0        0         0  ...       0     0              0   
3             0       0       35         0  ...       0     0              0   
4             0       0        2         0  ...       0     0              0   

   quiz  repeatactivity  resource  sharedsubpage  subpage  url  final_result  
0

In [ ]:
# Tính tổng số lần nhấp chuột theo từng tuần
weekly_clicks_summary = pass_df.groupby('week').sum().reset_index()

# Hiển thị DataFrame kết quả
print(weekly_clicks_summary)


    week                                        code_module  \
0     -3  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...   
1     -2  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...   
2     -1  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...   
3      0  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...   
4      1  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...   
5      2  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...   
6      3  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...   
7      4  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...   
8      5  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...   
9      6  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...   
10     7  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...   
11     8  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...   
12     9  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...   
13    10  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...   
14    11  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

In [ ]:
# Lọc dữ liệu chỉ lấy các tuần từ 1 đến 5
first_5_weeks_df = sum_click_by_activity_by_week[(sum_click_by_activity_by_week['week'] <= 5) & (sum_click_by_activity_by_week['week'] >= 1)]

# Nhóm dữ liệu theo 'code_module', 'code_presentation', 'id_student' và tính tổng số lần nhấp chuột
first_5_weeks_summary = first_5_weeks_df.groupby(['code_module', 'code_presentation', 'id_student']).sum().reset_index()
first_5_weeks_summary = first_5_weeks_summary.drop(columns=['week','final_result'])

first_5_weeks_summary = first_5_weeks_summary.merge(df_studentinfo[['code_module','code_presentation','id_student', 'final_result']], on=['code_module', 'code_presentation', 'id_student'], how='left')
# Hiển thị DataFrame kết quả
print(first_5_weeks_summary.head())
output_csv_path='./first_5_weeks_summary.csv'
first_5_weeks_summary.to_csv(output_csv_path, index=False)

  code_module code_presentation  id_student  dataplus  dualpane  externalquiz  \
0         AAA             2013J       11391         0         0             0   
1         AAA             2013J       28400         0         0             0   
2         AAA             2013J       30268         0         0             0   
3         AAA             2013J       31604         0         0             0   
4         AAA             2013J       32885         0         0             0   

   folder  forumng  glossary  homepage  ...  ouwiki  page  questionnaire  \
0       0       40         0        35  ...       0     0              0   
1       0      108         0        81  ...       0     0              0   
2       0       87         0        27  ...       0     0              0   
3       0      121         0        87  ...       0     0              0   
4       0       65         0        53  ...       0     0              0   

   quiz  repeatactivity  resource  sharedsubpage  subpag

In [ ]:
# Lọc dữ liệu chỉ lấy các tuần từ 1 đến 10
first_10_weeks_df = sum_click_by_activity_by_week[(sum_click_by_activity_by_week['week'] <= 10) & (sum_click_by_activity_by_week['week'] >= 1)]

# Nhóm dữ liệu theo 'code_module', 'code_presentation', 'id_student' và tính tổng số lần nhấp chuột
first_10_weeks_summary = first_10_weeks_df.groupby(['code_module', 'code_presentation', 'id_student']).sum().reset_index()
first_10_weeks_summary = first_10_weeks_summary.drop(columns=['week','final_result'])

first_10_weeks_summary = first_10_weeks_summary.merge(df_studentinfo[['code_module','code_presentation','id_student', 'final_result']], on=['code_module', 'code_presentation', 'id_student'], how='left')
# Hiển thị DataFrame kết quả
print(first_10_weeks_summary.head())
output_csv_path='./first_10_weeks_summary.csv'
first_10_weeks_summary.to_csv(output_csv_path, index=False)

  code_module code_presentation  id_student  dataplus  dualpane  externalquiz  \
0         AAA             2013J       11391         0         0             0   
1         AAA             2013J       28400         0         0             0   
2         AAA             2013J       30268         0         0             0   
3         AAA             2013J       31604         0         0             0   
4         AAA             2013J       32885         0         0             0   

   folder  forumng  glossary  homepage  ...  ouwiki  page  questionnaire  \
0       0       81         0        53  ...       0     0              0   
1       0      164         0       113  ...       0     0              0   
2       0       87         0        27  ...       0     0              0   
3       0      206         0       169  ...       0     0              0   
4       0       65         0        59  ...       0     0              0   

   quiz  repeatactivity  resource  sharedsubpage  subpag

In [ ]:
# Lọc dữ liệu chỉ lấy các tuần từ 1 đến 20
first_20_weeks_df = sum_click_by_activity_by_week[(sum_click_by_activity_by_week['week'] <= 20) & (sum_click_by_activity_by_week['week'] >= 1)]

# Nhóm dữ liệu theo 'code_module', 'code_presentation', 'id_student' và tính tổng số lần nhấp chuột
first_20_weeks_summary = first_20_weeks_df.groupby(['code_module', 'code_presentation', 'id_student']).sum().reset_index()
first_20_weeks_summary = first_20_weeks_summary.drop(columns=['week','final_result'])

first_20_weeks_summary = first_20_weeks_summary.merge(df_studentinfo[['code_module','code_presentation','id_student', 'final_result']], on=['code_module', 'code_presentation', 'id_student'], how='left')
# Hiển thị DataFrame kết quả
print(first_20_weeks_summary.head())
output_csv_path='./first_20_weeks_summary.csv'
first_20_weeks_summary.to_csv(output_csv_path, index=False)

  code_module code_presentation  id_student  dataplus  dualpane  externalquiz  \
0         AAA             2013J       11391         0         0             0   
1         AAA             2013J       28400         3         0             0   
2         AAA             2013J       30268         0         0             0   
3         AAA             2013J       31604         0         0             0   
4         AAA             2013J       32885         0         0             0   

   folder  forumng  glossary  homepage  ...  ouwiki  page  questionnaire  \
0       0      129         0        89  ...       0     0              0   
1       0      232         0       180  ...       0     0              0   
2       0       87         0        27  ...       0     0              0   
3       0      332         0       273  ...       0     0              0   
4       0       89         0       103  ...       0     0              0   

   quiz  repeatactivity  resource  sharedsubpage  subpag

In [ ]:
# Lọc dữ liệu chỉ lấy các tuần từ 1 đến 30
first_30_weeks_df = sum_click_by_activity_by_week[(sum_click_by_activity_by_week['week'] <= 30) & (sum_click_by_activity_by_week['week'] >= 1)]

# Nhóm dữ liệu theo 'code_module', 'code_presentation', 'id_student' và tính tổng số lần nhấp chuột
first_30_weeks_summary = first_30_weeks_df.groupby(['code_module', 'code_presentation', 'id_student']).sum().reset_index()
first_30_weeks_summary = first_30_weeks_summary.drop(columns=['week','final_result'])

first_30_weeks_summary = first_30_weeks_summary.merge(df_studentinfo[['code_module','code_presentation','id_student', 'final_result']], on=['code_module', 'code_presentation', 'id_student'], how='left')
# Hiển thị DataFrame kết quả
print(first_30_weeks_summary.head())
output_csv_path='./first_30_weeks_summary.csv'
first_30_weeks_summary.to_csv(output_csv_path, index=False)

  code_module code_presentation  id_student  dataplus  dualpane  externalquiz  \
0         AAA             2013J       11391         0         0             0   
1         AAA             2013J       28400        10         0             0   
2         AAA             2013J       30268         0         0             0   
3         AAA             2013J       31604         2         0             0   
4         AAA             2013J       32885         0         0             0   

   folder  forumng  glossary  homepage  ...  ouwiki  page  questionnaire  \
0       0      141         0       111  ...       0     0              0   
1       0      286         0       251  ...       0     0              0   
2       0       87         0        27  ...       0     0              0   
3       0      416         0       350  ...       0     0              0   
4       0       91         2       124  ...       0     0              0   

   quiz  repeatactivity  resource  sharedsubpage  subpag

In [ ]:
# Lọc dữ liệu chỉ lấy các tuần từ 1 đến 39
first_39_weeks_df = sum_click_by_activity_by_week[(sum_click_by_activity_by_week['week'] <= 39) & (sum_click_by_activity_by_week['week'] >= 1)]

# Nhóm dữ liệu theo 'code_module', 'code_presentation', 'id_student' và tính tổng số lần nhấp chuột
first_39_weeks_summary = first_39_weeks_df.groupby(['code_module', 'code_presentation', 'id_student']).sum().reset_index()
first_39_weeks_summary = first_39_weeks_summary.drop(columns=['week','final_result'])

first_39_weeks_summary = first_39_weeks_summary.merge(df_studentinfo[['code_module','code_presentation','id_student', 'final_result']], on=['code_module', 'code_presentation', 'id_student'], how='left')
# Hiển thị DataFrame kết quả
print(first_39_weeks_summary.head())
output_csv_path='./first_39_weeks_summary.csv'
first_39_weeks_summary.to_csv(output_csv_path, index=False)

  code_module code_presentation  id_student  dataplus  dualpane  externalquiz  \
0         AAA             2013J       11391         0         0             0   
1         AAA             2013J       28400        10         0             0   
2         AAA             2013J       30268         0         0             0   
3         AAA             2013J       31604         2         0             0   
4         AAA             2013J       32885         0         0             0   

   folder  forumng  glossary  homepage  ...  ouwiki  page  questionnaire  \
0       0      176         0       121  ...       0     0              0   
1       0      316         0       268  ...       0     0              0   
2       0       87         0        27  ...       0     0              0   
3       0      605         0       402  ...       0     0              0   
4       0      121         2       152  ...       0     0              0   

   quiz  repeatactivity  resource  sharedsubpage  subpag

Xem thông tin về data